<a href="https://colab.research.google.com/github/Yatharth19/Tf-idf-dod/blob/main/Tf_idf_dod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Dependencies

In [ ]:

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import csv

Reading the dataset

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/cancer_clinical_trials_retry2_resized (4).csv')


Checking the dataframe

In [ ]:
print(df.shape)

print(df.head(10))

In [ ]:
# Checking the outcomes in the dataset

df['qualification'].value_counts()

Checking for NULL values

In [ ]:
df.isnull().sum()

In [ ]:
# Checking the number of unique studies conducted in our dataset

df['study'].unique().size

In [ ]:
# Function to convert the study conducted phrase to be more meaningful

def convert(str):
  new_str=str.replace("study interventions are ","")
  return new_str


Creating the combination for prediction using both study and condition

In [ ]:
df['study']=(df['study'].apply(convert))
#Splitting string into list of words
df['study_words']=df['study'].apply(lambda x: x.split())
df['condition_words']=df['condition'].apply(lambda x: x.split())

df['study'].value_counts()

# Converting into lowercase
df['study']=df['study'].apply(lambda x:x.lower())

df.head()

#Removing space from the study name
df['study_words']=df['study_words'].apply(lambda x:[i.replace(" ","") for i in x])
df['condition_words']=df['condition_words'].apply(lambda x:[i.replace(" ","") for i in x])

df.head()

df['keys']=df['study_words']+df['condition_words']

df['keys'].head()

#Convert this list into a string
df['keys']=df['keys'].apply(lambda x:" ".join(x))

df.head()

# Converting the text into lowercase to remove any ambiguity

df['keys']=df['keys'].apply(lambda x:x.lower())



Tf-Idf Vectorizer

In [ ]:
# Tf-Idf vectorizer

test=df['keys']

test.shape

tfidf=TfidfVectorizer()
response=tfidf.fit_transform(test)

similarity=cosine_similarity(response)

df.head()

df['study']=df['study'].apply(lambda x:x.rstrip())

Predictor Function

In [ ]:

error=0
def predict(str1,str2):
  strin=convert(str1).lower()+str2.lower()
  str1=convert(str1)
  
  try:
    index=df[df['keys']==strin].index[0]
    distance=similarity[index]

    dis_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    maxval=0
    for i in dis_list:
      if(df.iloc[i[0]].qualification==1):
        maxval=maxval+df.iloc[i[0]].qualification
      else:
        maxval-=1


    
    if maxval>0:
      # print("1")
      current_prediction=1
    else:
      # print("0")
      current_prediction=0
    return current_prediction
  except:
    # print("error")
    global error
    error+=1



Checking the model

In [ ]:
study=list(df.study)
condition=list(df.condition)
qualification=list(df.qualification)
correct=0

total=[[]]
for i in range(0,20000):
  str1=study[i]
  str2=condition[i]

  current_prediction=predict(str1.lower(),str2.lower())
  if qualification[i]==current_prediction:
    correct=correct+1
    
print(correct)
global error
print("error=>",error)
print("Accuracy is->",(correct/20000)*100)
